# Order Analysis

# TODO: Plot order filled and order cancelled and hedges?

In [150]:
import plotly.figure_factory as ff
import pandas as pd
import plotly.express as px
import os
import plotly.graph_objects as go
import numpy as np

In [17]:
def plot_price(csv):
    """Plots a line chart of a given price timeseries. 2 lines, one etf the other future"""
    df = pd.read_csv(csv)
    fig = px.line(df, y='price', color='instrument', title=csv)
    fig.show()

def plot_diff(csv):
    """Plots a line chart of etf - future"""
    df = pd.read_csv(csv)
    fig = go.Figure()
    diffs = df[df['instrument']=='etf']['price'].reset_index(drop=True) - df[df['instrument']=='future']['price'].reset_index(drop=True)
    fig.add_trace(go.Scatter(y=diffs))
    fig.add_trace(go.Scatter(y=[0]*len(diffs)))
    fig.update_layout(title=f"{csv}-DIFFS")
    fig.show()

def csv_paths(number) -> str:
    """Returns path given market data number"""
    return "data/" + [file for file in os.listdir('data') if file.startswith(f"data_{number}")][0]
    

In [19]:
plot_price(csv_paths('t2'))

In [135]:
def plot_orders(events_csv, trader):
    """Just etf orders - may be filled or unfilled
    Size of point is an indication of volume
    """
    title = f"{os.path.basename(events_csv).split('_')[0]} - {trader}"
    events_df = pd.read_csv(events_csv)
    orders = events_df[(events_df['Competitor']==trader) & events_df['Operation'].isin(['Insert'])]
    buys = orders[orders['Side']=='B']
    asks = orders[orders['Side']=='A']

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x = buys['Time'], y = buys['Price'], mode='markers', name='Buy Orders', marker=dict(size=[((4) * (x - 1) / 99) + 3 for x in buys['Volume']])
    ))
    fig.add_trace(go.Scatter(
        x = asks['Time'], y = asks['Price'], mode='markers', name='Ask Orders', marker=dict(size=[((4) * (x - 1) / 99) + 3 for x in asks['Volume']])
    ))
    
    # fig.update_traces(marker={'size': 4})
    fig.update_layout(title=title, yaxis_title='Price', xaxis_title='Timestep')
    fig.show()



In [137]:
m30_path = 'data/tournament2_files/match30_events.csv'

plot_orders(m30_path, 'AP_TRADING')

In [130]:
def plot_fills(events_csv, trader):
    """Just etf orders - may be filled or unfilled
    Size of point is an indication of volume
    """
    title = f"{os.path.basename(events_csv).split('_')[0]} - {trader}"
    events_df = pd.read_csv(events_csv)
    orders = events_df[(events_df['Competitor']==trader) & events_df['Operation'].isin(['Insert'])]
    buys = orders[orders['Side']=='B']
    asks = orders[orders['Side']=='A']

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x = buys['Time'], y = buys['Price'], mode='markers', name='Buy Orders', marker=dict(size=[((4) * (x - 1) / 99) + 3 for x in buys['Volume']])
    ))
    fig.add_trace(go.Scatter(
        x = asks['Time'], y = asks['Price'], mode='markers', name='Ask Orders', marker=dict(size=[((4) * (x - 1) / 99) + 3 for x in asks['Volume']])
    ))
    
    # fig.update_traces(marker={'size': 4})
    fig.update_layout(title=title, yaxis_title='Price', xaxis_title='Timestep')
    fig.show()


0.04040404040404041

## Match 30 Analysis

In [29]:
m30 = pd.read_csv('data/tournament2_files/match30_events.csv')

In [28]:
omar = m30[m30['Competitor']=='OneManArmyReturns']
omar

,Time,Competitor,Operation,OrderId,Instrument,Side,Volume,Price,Lifespan,Fee
14,0.002096,OneManArmyReturns,Insert,2,1.0,B,50,65700.0,G,NaN
15,0.002256,OneManArmyReturns,Insert,3,1.0,A,49,66000.0,G,NaN
485,0.254879,OneManArmyReturns,Cancel,2,NaN,NaN,-50,NaN,NaN,NaN
486,0.255951,OneManArmyReturns,Cancel,3,NaN,NaN,-49,NaN,NaN,NaN
487,0.256481,OneManArmyReturns,Insert,4,1.0,B,50,65800.0,G,NaN
...,...,...,...,...,...,...,...,...,...,...
1709317,3599.502111,OneManArmyReturns,Insert,29589,1.0,A,97,59200.0,G,NaN
1709389,3599.751094,OneManArmyReturns,Cancel,29588,NaN,NaN,-2,NaN,NaN,NaN
1709390,3599.751109,OneManArmyReturns,Cancel,29589,NaN,NaN,-97,NaN,NaN,NaN
1709391,3599.751119,OneManArmyReturns,Insert,29590,1.0,B,2,58400.0,G,NaN


The hedges and trades are always equal....

In [201]:
def analyse_operations(df: pd.DataFrame):
    """Outputs a bar plot of each trader operations
    Args:
        df: a df of match events data
    """
    ops = ['Insert', 'Cancel', 'Trade']
    operations = []
    for trader in df['Competitor'].unique():
        if trader is not np.nan:
            op_counts = list(df[df['Competitor']==trader]['Operation'].value_counts())[0:3]
            operations.append(go.Bar(
                name=trader, x=ops, y=op_counts
            ))
    
    fig = go.Figure(data = operations)
    fig.update_layout(barmode='group')
    fig.show()

def analyse_volumes(df, traders:list):
    """See distribution of volumes (for just orders)
    traders: max 3 or error"""
    # Group data together
    vol_data = [list(df[(df['Competitor']==trader) & (df['Operation'] == 'Insert')]['Volume']) for trader in traders]

    # Create distplot with custom bin_size
    fig = ff.create_distplot(vol_data, traders, bin_size=.2)
    fig.show()

In [60]:
analyse_operations(m30)

We need to make far more inserts, cancels and trades, sufficed to say, the others are probably actually market making xd

In [202]:
analyse_volumes(m30, ['AP_TRADING', 'OneManArmyReturns'])

o.m.a.r has an even dist with more order vols = 0 and = 100 